In [103]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import AzureOpenAI
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import LLMChain, APIChain
from langchain.chains.api.prompt import API_RESPONSE_PROMPT
from langchain.schema import Document
from langchain.tools import BaseTool
from langchain.agents import ZeroShotAgent, AgentExecutor, AgentType, initialize_agent, Tool, load_tools
from langchain.agents.agent_toolkits.openapi import planner
from langchain.requests import TextRequestsWrapper
from functools import reduce
from langchain.chains.api import open_meteo_docs
from datetime import datetime, timedelta
import requests
import pandas as pd
import os
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient

search_endpoint = 'https://search-service01.search.windows.net'
search_key = '73Swa5YqUR5IRMwUIqOH6ww2YBm3SveLv7rDmZVXtIAzSeBjEQe9'
private_index_name = "nois-private-v3-index"

retriever_private = SearchClient(
    endpoint=search_endpoint,
    index_name=private_index_name,
    credential=AzureKeyCredential(search_key),
    b=0.0,
    k1=0.3,
    searchMode="all"
)

In [8]:
import tiktoken

enc = tiktoken.encoding_for_model("gpt-3.5-turbo")


def count_tokens(s):
    return len(enc.encode(s))
string = '''[{'id': '262aa143-307a-4b9f-848d-1e6bf2e5cacb', 'name': 'Hung Bui', 'email': 'hung.bui@nois.vn', 'fullName': 'BÙI TUẤN HƯNG', 'jobTitle': 'Lập trình viên', 'mobilePhone': '0966772170', 'birthday': '1997-08-17T00:00:00', 'offDay': 0.5, 'yearOffDay': 6, 'maxDayOff': 15, 'remainDayOffLastYear': 0, 'offDayUseRamainDayOffLastYear': 1, 'gender': 1, 'personalEmail': 'hungbt.dev@gmail.com', 'avatar': 'https://noishrm.blob.core.windows.net/avatar/8fec9283-e82f-43ec-8f61-addb732dabcb.jpeg', 'acceptOfferDate': '2022-10-31T00:00:00', 'quitJobDate': '', 'bonusDayOff': 0, 'managerId': '', 'managerFullName': '', 'departmentId': 0, 'departmentName': 'Development Unit 1', 'groupUserId': '', 'groupUserName': 'Nhân viên chính thức', 'hasLeft': '', 'identityCard': '', 'dateOfIssue': '2021-08-10T00:00:00', 'issuedBy': 'Cục trưởng CCS QLHC về TTXH', 'placeOfTemporaryResidence': '502/15/2 Thống Nhất, p.16, Gò Vấp, TP.HCM', 'placeOfResidence': '161 Đường 458, Ấp Hội Thạnh, Xã Trung An, Huyện Củ Chi, TP.HCM', 'relativeName': 'Lê Thị Kiều', 'relationship': 'Mẹ', 'relativeMobilePhone': '', 'hasBankAccount': '', 'bankName': 'Vietcombank', 'bankBranch': 'DONG SAI GON-PGD THANH DA', 'bankAccountNumber': '0531002579897', 'birthPlace': 'Ấp Hội Thạnh, Xã Trung An, Huyện Củ Chi, TP.HCM', 'isFirstLogin': '', 'roles': '', 'taxIdentificationNumber': '8460719408', 'education': '', 'jobId': '', 'jobName': 'Developer', 'levelId': '', 'levelName': ''}, {'id': '72fb1693-9cc1-4923-b0a4-afb18469613c', 'name': 'KHASNH BUIF', 'email': '', 'fullName': '', 'jobTitle': '', 'mobilePhone': '', 'birthday': '', 'offDay': 1, 'yearOffDay': 0, 'maxDayOff': 0, 'remainDayOffLastYear': 0, 'offDayUseRamainDayOffLastYear': 0, 'gender': 0, 'personalEmail': '', 'avatar': '', 'acceptOfferDate': '', 'quitJobDate': '', 'bonusDayOff': 0, 'managerId': '', 'managerFullName': '', 'departmentId': 0, 'departmentName': '', 'groupUserId': '', 'groupUserName': '', 'hasLeft': '', 'identityCard': '', 'dateOfIssue': '', 'issuedBy': '', 'placeOfTemporaryResidence': '', 'placeOfResidence': '', 'relativeName': '', 'relationship': '', 'relativeMobilePhone': '', 'hasBankAccount': '', 'bankName': '', 'bankBranch': '', 'bankAccountNumber': '', 'birthPlace': '', 'isFirstLogin': '', 'roles': '', 'taxIdentificationNumber': '', 'education': '', 'jobId': '', 'jobName': '', 'levelId': '', 'levelName': ''}, {'id': 'c8604118-7db6-4e08-8926-5bd8c81bce81', 'name': 'Bao Ho', 'email': 'honguyenngocbao@gmail.com', 'fullName': 'HỒ NGUYỄN NGỌC BẢO', 'jobTitle': '', 'mobilePhone': '', 'birthday': '', 'offDay': 0, 'yearOffDay': 0, 'maxDayOff': 0, 'remainDayOffLastYear': 0, 'offDayUseRamainDayOffLastYear': 0, 'gender': 0, 'personalEmail': '', 'avatar': '', 'acceptOfferDate': '', 'quitJobDate': '', 'bonusDayOff': 0, 'managerId': '', 'managerFullName': '', 'departmentId': 0, 'departmentName': '', 'groupUserId': '', 'groupUserName': '', 'hasLeft': '', 'identityCard': '', 'dateOfIssue': '', 'issuedBy': '', 'placeOfTemporaryResidence': '', 'placeOfResidence': '', 'relativeName': '', 'relationship': '', 'relativeMobilePhone': '', 'hasBankAccount': '', 'bankName': '', 'bankBranch': '', 'bankAccountNumber': '', 'birthPlace': '', 'isFirstLogin': '', 'roles': '', 'taxIdentificationNumber': '', 'education': '', 'jobId': '', 'jobName': '', 'levelId': '', 'levelName': ''}]'''

print(count_tokens(string))

1171


In [13]:
# Load up your LLM
llm = AzureChatOpenAI(
    openai_api_type="azure",
    openai_api_base='https://openai-nois-intern.openai.azure.com/',
    openai_api_version="2023-03-15-preview",
    deployment_name='gpt-35-turbo',
    openai_api_key='400568d9a16740b88aff437480544a39',
    temperature=0.5,
    max_tokens=450
)

llm2 = AzureChatOpenAI(
    openai_api_type="azure",
    openai_api_base='https://openai-nois-intern.openai.azure.com/',
    openai_api_version="2023-03-15-preview",
    deployment_name='gpt-35-turbo',
    openai_api_key='400568d9a16740b88aff437480544a39',
    temperature=0,
    max_tokens=600,
    stop=['\n', '<|im_end|>', '<|im_sep|>']
)

llm3 = AzureChatOpenAI(
    openai_api_type="azure",
    openai_api_base='https://openai-nois-intern.openai.azure.com/',
    openai_api_version="2023-03-15-preview",
    deployment_name='gpt-35-turbo-16k',
    openai_api_key='400568d9a16740b88aff437480544a39',
    temperature=0.0,
    max_tokens=600,
    top_p = 0.75
)

history = []
history_entry = 1

def get_history_as_txt():
    txt = ""
    hist = history[::-1]
    
    for i in hist[:3]:
        txt += f"\n<|im_start|>user\n{i['user']}\n<|im_end|>\n"
        txt += f"<|im_start|>assistant\n{i['AI']}\n<|im_end|>"
        
    return txt

def add_to_history(user_msg, ai_msg):
    history.append({'user': user_msg, 'AI': ai_msg})
    
def save_history_to_file():
    global history_entry
    file = open(f"chatbot-cases/chat-{history_entry}.txt", 'w', encoding='utf-8')
    file.write(get_history_as_txt())
    
    file.close()
    
    history_entry += 1
    
def doc_to_str(doc):
    string = ""
    for i in doc:
        string += i.page_content
        
    return string

print(llm2)
doc = open_meteo_docs.OPEN_METEO_DOCS
print(llm2.get_num_tokens(doc))

WARNING! stop is not default parameter.
                    stop was transferred to model_kwargs.
                    Please confirm that stop is what you intended.
WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.


lc_kwargs={'openai_api_type': 'azure', 'openai_api_base': 'https://openai-nois-intern.openai.azure.com/', 'openai_api_version': '2023-03-15-preview', 'deployment_name': 'gpt-35-turbo', 'openai_api_key': '400568d9a16740b88aff437480544a39', 'temperature': 0, 'max_tokens': 600, 'stop': ['\n', '<|im_end|>', '<|im_sep|>']} verbose=False callbacks=None callback_manager=None client=<class 'openai.api_resources.chat_completion.ChatCompletion'> model_name='gpt-3.5-turbo' temperature=0.0 model_kwargs={'stop': ['\n', '<|im_end|>', '<|im_sep|>']} openai_api_key='400568d9a16740b88aff437480544a39' openai_api_base='https://openai-nois-intern.openai.azure.com/' openai_organization='' openai_proxy='' request_timeout=None max_retries=6 streaming=False n=1 max_tokens=600 deployment_name='gpt-35-turbo' openai_api_type='azure' openai_api_version='2023-03-15-preview'
761


In [4]:
url = 'https://hrm-nois-fake.azurewebsites.net'

user_id = 'a27ef124-989c-4e88-be05-1a00dd9075e8'

email = 'hung.bui@nois.vn'

obj = {
    'id': user_id,
    'levelName': 'CEO'
}

endpoint = f'/api/User/{user_id}' if user_id else '/api/User'
print(endpoint)

reply = requests.put(url + endpoint, data=obj)

print(reply.text)

/api/User/a27ef124-989c-4e88-be05-1a00dd9075e8
{"type":"https://tools.ietf.org/html/rfc7231#section-6.5.13","title":"Unsupported Media Type","status":415,"traceId":"00-3d38000af8e0e0c6ef297ff755482ce9-7e27477d438b0d0a-00"}


In [14]:
# print(open_meteo_docs.OPEN_METEO_DOCS)
hrm_doc = open("HRM_API_DOC.txt", 'r', encoding='utf-8')
api_doc = hrm_doc.read()
print(api_doc)
print(f"API token length: {count_tokens(api_doc)}")

BASE URL: https://hrm-nois-fake.azurewebsites.net/

HRM system
API DOCUMENTATION
Endpoints:
- /api/User: does not accept any input. Use GET with this endpoint to get a list of users within the HRM system.
- /api/User/<id>: accepts an ID string as a path variable. Use GET with this endpoint along with a user's ID to get data of a specific user.
- /api/LeaveApplication/<userId>: accepts an ID string as a path variable. Use GET with this endpoint along with a user's ID to get all leave applications submitted by a user.
API token length: 128


In [15]:
chain_new = APIChain.from_llm_and_api_docs(llm2, api_doc, verbose=True)

In [18]:
print(chain_new)

lc_kwargs={'api_request_chain': LLMChain(lc_kwargs={'llm': AzureChatOpenAI(lc_kwargs={'openai_api_type': 'azure', 'openai_api_base': 'https://openai-nois-intern.openai.azure.com/', 'openai_api_version': '2023-03-15-preview', 'deployment_name': 'gpt-35-turbo', 'openai_api_key': '400568d9a16740b88aff437480544a39', 'temperature': 0, 'max_tokens': 600, 'stop': ['\n', '<|im_end|>', '<|im_sep|>']}, verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={'stop': ['\n', '<|im_end|>', '<|im_sep|>']}, openai_api_key='400568d9a16740b88aff437480544a39', openai_api_base='https://openai-nois-intern.openai.azure.com/', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=600, deployment_name='gpt-35-turbo', openai_api_type='azure', openai_api_version='2023-03-15-preview'), 'prompt': PromptTemplate(lc_kwargs={'input_var

In [22]:
chain_new("Users's birthday'?")

Error in on_chain_start callback: 'name'


https://hrm-nois-fake.azurewebsites.net/api/User
{"message":"OK","data":[{"id":"262aa143-307a-4b9f-848d-1e6bf2e5cacb","name":"Hung Bui","email":"hung.bui@nois.vn","fullName":"BÙI TUẤN HƯNG","jobTitle":"Lập trình viên","mobilePhone":"0966772170","birthday":"1997-08-17T00:00:00","offDay":0.5,"yearOffDay":6,"maxDayOff":15,"remainDayOffLastYear":0,"offDayUseRamainDayOffLastYear":1,"gender":1,"personalEmail":"hungbt.dev@gmail.com","avatar":"https://noishrm.blob.core.windows.net/avatar/8fec9283-e82f-43ec-8f61-addb732dabcb.jpeg","acceptOfferDate":"2022-10-31T00:00:00","quitJobDate":"","bonusDayOff":0,"managerId":"","managerFullName":"","departmentId":0,"departmentName":"Development Unit 1","groupUserId":"","groupUserName":"Nhân viên chính thức","hasLeft":"","identityCard":"","dateOfIssue":"2021-08-10T00:00:00","issuedBy":"Cục trưởng CCS QLHC về TTXH","placeOfTemporaryResidence":"502/15/2 Thống Nhất, p.16, Gò Vấp, TP.HCM","placeOfResidence":"161 Đường 458, Ấp Hội Thạnh, Xã Trung An, Huyện Củ C

{'question': "Users's birthday'?",
 'output': "The response from the API provides information about two users. The first user's name is Hung Bui and their birthday is on August 17, 1997. The second user's name is Bao Ho, but their birthday is not provided."}

In [127]:
def get_users(query: str = None):
    return requests.get(url + '/api/User').json()['data']

def get_user_through_email(email):
    response = requests.get(url + f'/api/User/{email}')
    if response.status_code == 200:
        return response.json()['data']
    
    return f'Error: {response.status_code}'

def get_user_by_id(ID: str):
    reply = requests.get(url + f'/api/User/{ID}')
    if not reply.json()['data']:
        return "Wrong ID, recheck your input."
    
    return reply.json()['data']

# def get_user_by_email(email: str):
#     reply = requests.get(url + f'/api/User').json()['data']
#     email = email.strip()
    
#     for i in reply:
#         if i['email'] == email:
#             return i
        
#     return "User not found. Recheck your inputs or ask the user for info."

def check_manager_name(name):
    response = requests.get(url + '/api/User/manager-users').json()['data']
    
    for data in response:
        if data['fullName'] == name: 
            return True
    return False

def post_method(idd, name, day):
    response = requests.post('https://hrm-nois-fake.azurewebsites.net/api/LeaveApplication/Submit', json = {
        "userId": idd,
        "reviewUserId": name,
        "relatedUserId": "string",
        "fromDate": "2023-07-08T09:12:04.799Z",
        "toDate": "2023-07-09T09:12:04.799Z",
        "leaveApplicationTypeId": 0,
        "leaveApplicationNote": "string",
        "periodType": 0,
        "numberOffDay": day
    })
    return response
    
lst = []
def submitLeaveApplication(args: str):
    global lst
    lst = args.split(', ')
    if len(lst) != 3:
        return "Incorrect number of arguments, this function requires 3 arguments: user's id, reviewer's id and number of days off."
    
    if '-' not in lst[0]:
        return "Incorrect ID. You can find the correct ID by using the HRM get by email tool."
    elif check_manager_name(lst[1]) != True:
        return "You must ask the user to provide the manager's name correctly."
    elif not lst[2].isdigit():
        return "Leave days must be a number. Ask the user if they didn't provide you with a number."
    
    lst[2] = int(lst[2])
    
#     print("\nUserId: ", lst[0])
#     print("ReviewerId: ", lst[1])
#     print("Number of days: ", lst[2])
    
#     currentDate = datetime.strptime(str(datetime.now()), '%Y-%m-%d')
#     toDate = currentDate + timedelta(days=lst[2])
    
#     reply = requests.post('https://hrm-nois-fake.azurewebsites.net/api/LeaveApplication/Submit', json = {
#         'UserId' : lst[0],
#         'reviewUserId' : lst[1],
#         'numberOffDay' : lst[2],
#         'fromDate': str(currentDate),
#         'toDate': str(toDate)
#     })
    
    
#     post_method(lst[0], lst[1], lst[2])
    
    return 'Successfully.'
        
tools = [
    Tool(
        name='HRM get all',
        func=get_users,
        description='useful for getting details of all users. No input necessary.'
    ),
    
    Tool(
        name='HRM get by id',
        func=get_user_by_id,
        description='useful for getting details of a specific user by ID. Input is a user\'s Id.'
    ),
    
    Tool(
        name='HRM get by email',
        func=get_user_by_email,
        description='useful for getting details of a specific user by email. Input is a user\'s email.'
    ),
    
    Tool(
        name='HRM submit leave',
        func=submitLeaveApplication,
        description='''useful for submitting leave applications for a specific user.
Input must include 3 parameters concatenated into a string separated by a comma and space, which are: 
1. user's id: you get by email. 
2. the name of manager who approve user's application: you must ask user to provide.
3. number of leave days: you must ask user to provide. 

'''
    )
]

# Example: 65255df1-d4c3-479b-a4b8-d63cb3db2beb, LÝ MINH QUÂN, 4

human = load_tools(['human'])

tools.extend(human)

'''If you cannot answer by using only 1 tool, try using other provided tools. If you cannot get any results from tools, say you don't know. Do not use any other sources other than the ones you obtain from tools.'''

prefix = """You are an intelligent assistant helping users interact with system called HRM through the use of tools.
Answer in the user's language.
The user chatting with you is named Nghiem Vo, with email nghiem.vo@nois.vn
You have access to the following tools:"""

suffix = """Begin!

Do not put ` at the beginning or the end of your outputs.
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "agent_scratchpad"],
)

print(prompt)

llm_chain = LLMChain(llm=llm3, prompt=prompt)

agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True, stop=["\nObservation:", "<|im_end|>", "<|im_sep|>"])
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors="Check your output and make sure it conforms! Get rid of the beginning and end ` in your output."
)

lc_kwargs={'template': "You are an intelligent assistant helping users interact with system called HRM through the use of tools.\nAnswer in the user's language.\nThe user chatting with you is named Nghiem Vo, with email nghiem.vo@nois.vn\nYou have access to the following tools:\n\nHRM get all: useful for getting details of all users. No input necessary.\nHRM get by id: useful for getting details of a specific user by ID. Input is a user's Id.\nHRM get by email: useful for getting details of a specific user by email. Input is a user's email.\nHRM submit leave: useful for submitting leave applications for a specific user.\nInput must include 3 parameters concatenated into a string separated by a comma and space, which are: \n1. user's id: you get by email. \n2. the name of manager who approve user's application: you must ask user to provide.\n3. number of leave days: you must ask user to provide. \n\n\nHuman: You can ask a human for guidance when you think you got stuck or you are not su

In [128]:
answer = agent_chain.run("I'd like to submit a leave application.")

Error in on_chain_start callback: 'name'


Thought: The user wants to submit a leave application.
Action: HRM submit leave
Action Input: nghiem.vo@nois.vn, [manager's name], [number of leave days]
Observation: Incorrect ID. You can find the correct ID by using the HRM get by email tool.
Thought:The user provided an incorrect ID. I need to use the HRM get by email tool to find the correct ID.
Action: HRM get by email
Action Input: nghiem.vo@nois.vn

KeyboardInterrupt: 

In [123]:
print(answer)
print(lst)

The leave application has been successfully submitted.
[]


In [129]:
print(post_method('65255df1-d4c3-479b-a4b8-d63cb3db2beb', 'LÝ MINH QUÂN', 1))

<Response [400]>
